In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.impute import SimpleImputer
import math

In [11]:
### Per Group TPRs ###
def groupRates(test_predictions, labels, sensitive):
    # True positives for z=1 and z=0
    TP_z1 = sum([l and p for (p,z,l) in zip(test_predictions,sensitive,labels) if z])
    FN_z1 = sum([l and not p for (p,z,l) in zip(test_predictions,sensitive,labels) if z])
    TP_z0 = sum([l and p for (p,z,l) in zip(test_predictions,sensitive,labels) if not z])
    FN_z0 = sum([l and not p for (p,z,l) in zip(test_predictions,sensitive,labels) if not z])
    # Convert to rates
    TPR1 = 0
    TPR0 = 0
    print(TP_z1, FN_z1, TP_z0, FN_z0)
    if TP_z1 > 0:
        TPR1 = TP_z1 / (TP_z1 + FN_z1)
    if TP_z0 > 0:
        TPR0 = TP_z0 / (TP_z0 + FN_z0)
    return TPR1, TPR0

In [12]:
###########################################
#Data Processing                          #
###########################################
df = pd.read_csv('bar_pass_prediction.csv')
df = df[['decile1b', 'decile3', 'lsat', 'ugpa', 'zfygpa', 'zgpa', 'fulltime', 'fam_inc', 'male', 'tier', 'race', 'pass_bar']]
df.replace(['?'] ,np.nan,inplace = True)
df['race'] = df['race'].apply(lambda x: 1 if x == 7 else 0)
sensitive = df['male'].to_numpy()
sensitive = sensitive == 1
print(np.sum(((~sensitive) & (df['pass_bar'] == 1))))
print(np.sum(((sensitive) & (df['pass_bar'] == 1))))
print(np.sum(df['pass_bar'] == 1))
num_pos = 5000
df_pos = df[df['pass_bar'] == 1].sample(num_pos, random_state=42)
df_neg = df[df['pass_bar'] == 0]
print(len(df_pos))
print(len(df_neg))
df = pd.concat([df_pos, df_neg])
df_noencode = pd.get_dummies(df, drop_first=True)
df_noencode = df_noencode


9263
11975
21238
5000
1169


In [13]:
###########################################
# Random Forest Classifer                 #
###########################################
X = df_noencode.drop('pass_bar', axis=1)
y = df_noencode['pass_bar']
sensitive = df['race'].to_numpy()
X_train, X_test, y_train, y_test, z_train, z_test = train_test_split(X, y, sensitive, test_size=0.2, random_state=42)
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print("Accuracy for random forest classifier:", accuracy_score(y_test, y_pred))
tpr1, tpr0 = groupRates(y_pred, y_test, z_test)
print("TPR1:", tpr1)
print("TPR0:", tpr0)

Accuracy for random forest classifier: 0.8476499189627229
817 36 113 45
TPR1: 0.9577960140679953
TPR0: 0.7151898734177216


In [14]:
###########################################
# Random Forest Classifer: Fine-tuning    #
###########################################
paramaters = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],      #this essientically just searches for most optimal paramters for RFC
    'min_samples_split': [2, 5]
}

crossV = KFold(n_splits=5, shuffle=True, random_state=42) #cross vadliation for parameter tuning
grid_search = GridSearchCV(
    estimator=RandomForestClassifier(random_state=42),
    param_grid=paramaters,    #gridsearch with cross validation from above
    cv=crossV,
    scoring='accuracy'
)
CrossValid = KFold(n_splits=5, shuffle=True, random_state=42) #final cross valid
grid_search = GridSearchCV(RandomForestClassifier(random_state=0), paramaters, cv=5, scoring='accuracy')
grid_search.fit(X_train, y_train)
print (grid_search.best_params_)
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
print("Accuracy for random forest classifier:", accuracy_score(y_test, y_pred))
tpr1, tpr0 = groupRates(y_pred, y_test, z_test)
print("TPR1:", tpr1)
print("TPR0:", tpr0)

{'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
Accuracy for random forest classifier: 0.8476499189627229
821 32 112 46
TPR1: 0.9624853458382181
TPR0: 0.7088607594936709


In [15]:
TwoCrossValid = cross_val_score(grid_search, X, y, cv=CrossValid, scoring='accuracy')

print("Accuracy for two nested cross validation:", (np.mean(TwoCrossValid), np.std(TwoCrossValid)))


Accuracy for two nested cross validation: (0.851190058375758, 0.00937018356458553)


In [ ]:
###########################################
# Pre-processing: Massaging               #
###########################################
def massage(data, S, b, label, plus):
    """Input: dataset D, sensitive attribute S, value b, and desired class up.
    Output: Classifier C, learned on massaged D"""

    w = 1 - b
    minus = 1 - plus

    # step 1: rank data for promotion and demotion lists
    pr, dem = rank(data, S, b, label, plus)
    n_pr = len(pr)
    n_dem = len(dem)
    print(f"num pr cands: {n_pr}")
    print(f"num dem cands: {n_dem}")

    # step 2: calculate number of data points to massage M
    group_b = data[data[S] == b]
    group_w = data[data[S] != b]
    plus_b = len(group_b[group_b[label] == plus])
    plus_w = len(group_w[group_w[label] == plus])
    n_b = len(group_b)
    n_w = len(group_w)
    n = len(data)
    disc = (plus_w / n_w) - (plus_b / n_b)
    M = disc * (n_b * n_w) / n
    M = int(M)
    print(f"M: {M}")

    # step 3: find top-M of pr and dem
    top_M_pr = pr.iloc[:M].index
    top_M_dem = dem.iloc[:M].index

    # step 4: change class labels of M-selected objects in each list
    massaged_data = data.copy()
    massaged_data.loc[top_M_pr, label] = plus
    massaged_data.loc[top_M_dem, label] = minus

    # step 5: train a classifier on massaged data
    X = massaged_data.drop(columns=[label])
    y = massaged_data[label]
    model = RandomForestClassifier(max_depth=10, min_samples_split=2, n_estimators=100, random_state=42)
    model.fit(X, y)

    return model

def rank(data, S, b, label, plus):
    """Input: dataset D, sensitive attribute S, and value b, desired class up.
    Output: promotion list pr, demotion list dem"""

    # step 1: learn a ranker for predicting positivity using the dataset as training set
    X = data.drop(label, axis=1)
    y = data[label]
    ranker = RandomForestClassifier(random_state=42)
    ranker.fit(X, y)
    scores = ranker.predict_proba(X)[:,1]
    scored_data = data.copy()
    scored_data['score'] = scores

    # step 2: build promotion and demotion lists
    pr = scored_data[(scored_data[S] == b) & (scored_data[label] != plus)]
    dem = scored_data[(scored_data[S] != b) & (scored_data[label] == plus)]

    # step 3: order promotion and demotion lists
    pr = pr.sort_values(by='score', ascending=False)
    dem = dem.sort_values(by='score', ascending=True)

    return pr, dem

# parameters
data = X_train.copy()
data['pass_bar'] = y_train
S = 'race'
b = 0 # white is 1, not white is 0
label = 'pass_bar'
plus = 1 # pass is 1, not pass is 0

# create classifier and predict
mass_clf = massage(data, S, b, label, plus)
mass_y_pred = mass_clf.predict(X_test)

# evaluate
print("Accuracy for massaged classifier:", accuracy_score(y_test, mass_y_pred))
tpr1, tpr0 = groupRates(mass_y_pred, y_test, z_test)
print("TPR1:", tpr1)
print("TPR0:", tpr0)

num pr cands: 428
num dem cands: 3416
M: 236
Accuracy for massaged classifier: 0.8192868719611021
770 83 148 10
TPR1: 0.902696365767878
TPR0: 0.9367088607594937


In [ ]:
###########################################
# In-Processing: Re-weighting             #
###########################################
def inProcessing(X_train, y_train, z_train):
    weights = []
    for y, z in zip(y_train, z_train):
      if y==1 and z==1:
          weights.append(2.8) #2.8
      elif y==1 and z==0:
          weights.append(0.75)
      elif y==0 and z==1:
          weights.append(0.85)
      else:
          weights.append(0.8)

    in_process_mod = clf.fit(X_train, y_train, sample_weight=weights)

    return in_process_mod


in_process_mod = inProcessing(X_train, y_train, z_train)
in_process_y_pred = in_process_mod.predict(X_test)
tpr1, tpr0 = groupRates(in_process_y_pred, y_test, z_test)

# Print results
print("TPR1:", tpr1)
print("TPR0:", tpr0)
print("Accuracy for Random Forest with In-Processing:", accuracy_score(y_test, in_process_y_pred))


810 43 118 40
TPR1: 0.9495896834701055
TPR0: 0.7468354430379747
Accuracy for Random Forest with In-Processing: 0.846839546191248


In [ ]:
###########################################
# Post-Processing: Affirmative Action     #
###########################################

def predict(probabilities, sensitive):
  w_threshold = 0.5
  nw_threshold = 0.20
  predictions = []
  for p, z in zip(probabilities, sensitive):
    if z == 1:
      predictions.append(p >= w_threshold)
    else:
      predictions.append(p >= nw_threshold)
  return predictions


prob = clf.predict_proba(X_test)

predictions = predict(prob[:, 1], z_test)


print("Accuracy for Random Forest with Post-Processing:", accuracy_score(y_test, predictions))
tpr1, tpr0 = groupRates(predictions, y_test, z_test)
print("TPR1:", tpr1)
print("TPR0:", tpr0)

Accuracy for Random Forest with Post-Processing: 0.8306320907617504
819 34 152 6
TPR1: 0.9601406799531067
TPR0: 0.9620253164556962


In [ ]:
###########################################
# Pre-Processing: Preferential Sampling   #
###########################################

def preferential_sampling(data, S, b, label, plus):
    """Input: dataset D, sensitive attribute S, and value b, desired class up.
    Output: Classifier C, learned on resampled D"""

    w = 1 - b
    minus = 1 - plus

    # num samples for groups
    n = len(data)
    n_b = len(data[data[S] == b])
    n_w = len(data[data[S] != b])
    n_plus = len(data[data[label] == plus])
    n_minus = len(data[data[label] == minus])
    n_DP = len(data[(data[S] == b) & (data[label] == plus)])
    n_DN = len(data[(data[S] == b) & (data[label] == minus)])
    n_FP = len(data[(data[S] != b) & (data[label] == plus)])
    n_FN = len(data[(data[S] != b) & (data[label] == minus)])

    # step 1: compute weights for groups
    W = {}
    W[(b, plus)] = (n_b * n_plus) / (n * n_DP)
    W[(b, minus)] = (n_b * n_minus) / (n * n_DN)
    W[(w, plus)] = (n_w * n_plus) / (n * n_FP)
    W[(w, minus)] = (n_w * n_minus) / (n * n_FN)

    # step 2: learn ranker for predicting positivity using the dataset as training set
    X = data.drop(columns=[label])
    y = data[label]
    ranker = RandomForestClassifier(max_depth=10, min_samples_split=2, n_estimators=100, random_state=42)
    ranker.fit(X, y)
    scores = ranker.predict_proba(X)[:, 1]
    scored_data = data.copy()
    scored_data['score'] = scores

    # step 3: split data into groups (disadvantaged vs favored and pos vs neg)
    DP = scored_data[(scored_data[S] == b) & (scored_data[label] == plus)]
    DN = scored_data[(scored_data[S] == b) & (scored_data[label] == minus)]
    FP = scored_data[(scored_data[S] != b) & (scored_data[label] == plus)]
    FN = scored_data[(scored_data[S] != b) & (scored_data[label] == minus)]

    # step 4: add ⌊(W(b, +)⌋ copies of DP
    DP_copies = math.floor(W[(b, plus)])
    DP_full = pd.concat([DP] * DP_copies, ignore_index=True)

    # step 5: add ⌊(W(b, +) - ⌊W(b, +)⌋) × |DP|⌋ lowest ranked elements of DP
    DP_extra = W[(b, plus)] - DP_copies
    n_DP_extra = int(math.floor(DP_extra * len(DP)))
    DP_extra = DP.sort_values(by='score', ascending=True).iloc[:n_DP_extra]
    resampled_DP = pd.concat([DP_full, DP_extra], ignore_index=True)

    # step 6: add ⌊W(b, −) × |DN|⌋ lowest ranked elements of DN
    n_DN_extra = int(math.floor(W[(b, minus)] * len(DN)))
    resampled_DN = DN.sort_values(by='score', ascending=True).iloc[:n_DN_extra]

    # step 7: add ⌊W(w, +) × |FP|⌋ highest ranked elements of FP
    n_FP_extra = int(math.floor(W[(w, plus)] * len(FP)))
    resampled_FP = FP.sort_values(by='score', ascending=False).iloc[:n_FP_extra]

    # step 8: add ⌊W(w, −)⌋ copies of FN
    FN_copies = math.floor(W[(w, minus)])
    FN_full = pd.concat([FN] * FN_copies, ignore_index=True)

    # step 9: add ⌊(W(w, −) - ⌊W(w, −)⌋) × |FN|⌋ highest ranked elements of FN
    FN_extra = W[(w, minus)] - FN_copies
    n_FN_extra = int(math.floor(FN_extra * len(FN)))
    FN_extra = FN.sort_values(by='score', ascending=False).iloc[:n_FN_extra]
    resampled_FN = pd.concat([FN_full, FN_extra], ignore_index=True)

    # step 10: combine resampled elements
    DPS = pd.concat([resampled_DP, resampled_DN, resampled_FP, resampled_FN], ignore_index=True)

    # step 11: train a classifier on preferentially sampled data
    X_DPS = DPS.drop(columns=[label, 'score'])
    y_DPS = DPS[label]
    model = RandomForestClassifier(max_depth=10, min_samples_split=2, n_estimators=100, random_state=42)
    model.fit(X_DPS, y_DPS)

    return model

# parameters
data = X_train.copy()
data['pass_bar'] = y_train
S = 'race'
b = 0 # white is 1, not white is 0
label = 'pass_bar'
plus = 1 # pass is 1, not pass is 0

# create classifier and predict
ps_clf = preferential_sampling(data, S, b, label, plus)
ps_y_pred = ps_clf.predict(X_test)

# evaluate
print("Accuracy for preferentially sampled classifier:", accuracy_score(y_test, ps_y_pred))
tpr1, tpr0 = groupRates(ps_y_pred, y_test, z_test)
print("TPR1:", tpr1)
print("TPR0:", tpr0)

Accuracy for preferentially sampled classifier: 0.8233387358184765
776 77 147 11
TPR1: 0.9097303634232122
TPR0: 0.930379746835443


In [ ]:
###########################################
# In-Processing: Grid Search              #
###########################################
from sklearn.model_selection import ParameterGrid

def inProcessing(X_train, y_train, sensitive_train):
    # Define weight ranges for the grid search
    weight_ranges = {
        'weight_z1_y1': [2.8],
        'weight_z0_y1': [.1, .5, .6, .71, .72, .74, .75],
        'weight_z1_y0': [.1, .5, .6,.85, .86, .87, .88,.9, 1, 1.2, 1.5],
        'weight_z0_y0': [ 0.8]
    }

    best_weights = None
    best_accuracy = 0
    best_tpr_diff = float('inf')

    # Perform grid search
    for weights_set in ParameterGrid(weight_ranges):
        weights = []
        for y, z in zip(y_train, sensitive_train):
            if z == 1 and y == 1:
                weights.append(weights_set['weight_z1_y1'])
            elif z == 0 and y == 1:
                weights.append(weights_set['weight_z0_y1'])
            elif z == 1 and y == 0:
                weights.append(weights_set['weight_z1_y0'])
            elif z == 0 and y == 0:
                weights.append(weights_set['weight_z0_y0'])

        in_process_mod = clf.fit(X_train, y_train, sample_weight=weights)
       # in_process_best_model = in_process_mod.best_estimator_
        in_process_y_pred = in_process_mod.predict(X_test)

        accuracy = accuracy_score(y_test, in_process_y_pred)
        tpr1, tpr0 = groupRates(in_process_y_pred, y_test, z_test)
        tpr_diff = abs(tpr1 - tpr0)

        print(f'weights {weights_set} and tpr {tpr1, tpr0}')
        if tpr_diff < best_tpr_diff:
            best_accuracy = accuracy
            best_tpr_diff = tpr_diff
            best_weights = weights_set

    print("Best Weights:", best_weights)
    print("Best Accuracy:", best_accuracy)
    print("Best TPR Difference:", best_tpr_diff)

    final_model = in_process_mod
    return final_model


inProcessing(X_train, y_train, z_train)

807 46 112 46
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0.1, 'weight_z1_y0': 0.1, 'weight_z1_y1': 2.8} and tpr (0.9460726846424384, 0.7088607594936709)
808 45 114 44
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0.1, 'weight_z1_y0': 0.5, 'weight_z1_y1': 2.8} and tpr (0.9472450175849941, 0.7215189873417721)
813 40 112 46
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0.1, 'weight_z1_y0': 0.6, 'weight_z1_y1': 2.8} and tpr (0.9531066822977726, 0.7088607594936709)
814 39 114 44
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0.1, 'weight_z1_y0': 0.85, 'weight_z1_y1': 2.8} and tpr (0.9542790152403282, 0.7215189873417721)
814 39 114 44
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0.1, 'weight_z1_y0': 0.86, 'weight_z1_y1': 2.8} and tpr (0.9542790152403282, 0.7215189873417721)
809 44 113 45
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0.1, 'weight_z1_y0': 0.87, 'weight_z1_y1': 2.8} and tpr (0.9484173505275498, 0.7151898734177216)
807 46 111 47
weights {'weight_z0_y0': 0.8, 'weight_z0_y1': 0

RandomForestClassifier(random_state=42)